In [1]:
import os, sys
global top_path  # the path of the top_level directory
global script_path, data_path, logging_path
top_path = '/scratch/user/zshuying/ppi_mutation/'
sys.path.append(top_path)
from scripts.utils import *
from scripts.utils_clinvar import *
pj=os.path.join
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [7]:
with open('/scratch/user/zshuying/ppi_mutation/data/ppi_huri_humap.txt','r') as f:
    interactions=eval(f.readline())

In [8]:
interactions

{'Q9BQ66-Q5TA78',
 'O14653-Q3KNW5',
 'Q3LI67-Q8WW24',
 'O95183-Q5JXX7',
 'Q8IUQ4-Q96HA8',
 'P54725-Q13114',
 'P49069-Q96AQ6',
 'P78362-P86480',
 'Q9H9E3-P25786',
 'Q02446-Q5MJ10',
 'Q53TS8-Q96RK4',
 'Q13625-Q5VVQ6',
 'Q14973-Q8N6F1',
 'A1L168-O95295',
 'O60216-Q13643',
 'A0A2R8Y4R9-P17482',
 'P35125-P36508',
 'P32242-Q12805',
 'Q13077-Q13526',
 'P62888-Q15050',
 'Q9UMX0-Q14055',
 'Q9BVG8-Q86V42',
 'Q15269-Q12788',
 'Q9NZD8-O14893',
 'Q5JTZ9-Q15257',
 'P02538-P55040',
 'Q96M91-Q8N3L3',
 'Q8IYA8-O00506',
 'A6NEM1-Q8WUI4',
 'Q6DKK2-Q15742',
 'Q9NXR5-Q15645',
 'Q96MV8-O60831',
 'Q00888-Q9UQ72',
 'Q53FD0-Q8WXW3',
 'P43364-Q9H875',
 'Q02878-O00425',
 'P15622-Q08379',
 'Q93062-Q9BTE1',
 'O15481-Q92752',
 'Q86UW9-Q12951',
 'Q9HBG6-O75386',
 'Q6P1W5-Q9Y3E2',
 'P16157-P54849',
 'Q8N6Y0-Q08AF8',
 'Q7RTU3-Q01196',
 'O75031-Q92805',
 'Q8NDC4-Q7Z3Z0',
 'P98095-Q96IQ9',
 'O43765-Q02747',
 'Q9Y584-A8MQ03',
 'Q9H4K1-Q9UHY1',
 'Q9UJX3-Q9H1A4',
 'P61254-P62280',
 'Q9UHG0-O94892',
 'Q4VCS5-Q9NRI5',
 'P552

In [10]:
exclude_self=[item for item in interactions if '-' in item]

In [11]:
interactions_eges=[(item.split('-')[0],item.split('-')[1]) for item in exclude_self]

In [12]:
interactions_eges

[('O43741', 'Q9P0T4'),
 ('Q9Y2R9', 'O60783'),
 ('O14958', 'Q9ULJ3'),
 ('Q96GF1', 'Q8NBI2'),
 ('Q13136', 'Q8NHQ1'),
 ('Q08117', 'Q9NWQ4'),
 ('O14879', 'Q96CS2'),
 ('P59817', 'Q86YH2'),
 ('Q9BYU1', 'Q5MJ09'),
 ('Q96LB9', 'Q9UHD4'),
 ('Q6UWI2', 'Q9UHD9'),
 ('Q8IUQ4', 'Q9UPY8'),
 ('Q7Z6G3', 'Q9NP79'),
 ('Q9UMS4', 'Q13435'),
 ('P31937', 'P21964'),
 ('Q93062', 'Q92567'),
 ('Q9NP66', 'Q96BS2'),
 ('Q12905', 'Q08211'),
 ('Q9H9Y6', 'O15160'),
 ('P18031', 'Q9NPL8'),
 ('O43561', 'Q8NBJ4'),
 ('Q9NPQ8', 'Q9Y371'),
 ('Q9UPZ6', 'Q9NY72'),
 ('Q8N1B4', 'Q6NYC8'),
 ('Q13630', 'P49903'),
 ('Q9BQY4', 'Q3LI64'),
 ('P61970', 'Q8IY57'),
 ('Q99459', 'Q08AF8'),
 ('Q86U28', 'Q9H5Z6'),
 ('Q92917', 'P31321'),
 ('P34810', 'Q96CS7'),
 ('O43395', 'Q9Y2I6'),
 ('P54105', 'P62308'),
 ('Q9UI95', 'Q9C0B1'),
 ('P86478', 'Q03989'),
 ('A8MQ03', 'Q9NVF9'),
 ('Q08379', 'Q96C55'),
 ('P55209', 'P19338'),
 ('Q9H2F3', 'Q9Y320'),
 ('Q93062', 'Q9BVN2'),
 ('Q9NW97', 'Q9H6H4'),
 ('Q7Z6R9', 'P0C7H8'),
 ('P83876', 'Q96D03'),
 ('Q17RD7',

In [12]:
import networkx as nx
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
G=nx.Graph(name='Protein Interaction Graph')


In [13]:
G.add_edges_from(interactions_eges)

In [14]:
print(G)

Graph named 'Protein Interaction Graph' with 11340 nodes and 72955 edges


In [15]:
print('Average degree', sum(dict(G.degree).values()) / len(G.nodes))


Average degree 12.866843033509701


In [16]:
12**4

20736

In [20]:
def merge_values_from_keys(dictionary, keys):
    merged_values = set()

    for key in keys:
        if key in dictionary:
            merged_values.update(dictionary[key])

    return merged_values
k_hop=pd.DataFrame(columns=['1-hop neighbors','2-hop neighbors','3-hop neighbors','4-hop neighbors','5-hop neighbors','6-hop neighbors'])
for n, nbrs in G.adj.items():
    k_hop.loc[n,'1-hop neighbors']= len(nbrs)
    hop2=merge_values_from_keys(G.adj,set(nbrs.keys()))
    hop12=hop2.union(set(nbrs.keys()))
    k_hop.loc[n,'2-hop neighbors']= len(hop12)
    hop3=merge_values_from_keys(G.adj,hop2)
    hop123=hop3.union(hop12)
    k_hop.loc[n,'3-hop neighbors']= len(hop123)
    hop4=merge_values_from_keys(G.adj,hop3)
    hop1234=hop4.union(hop123)
    k_hop.loc[n,'4-hop neighbors']= len(hop1234)  
    hop5=merge_values_from_keys(G.adj,hop4)
    hop12345=hop4.union(hop1234)
    k_hop.loc[n,'5-hop neighbors']= len(hop12345) 
    hop6=merge_values_from_keys(G.adj,hop5)
    hop123456=hop6.union(hop12345)
    k_hop.loc[n,'6-hop neighbors']= len(hop123456) 

In [63]:
k_hop

,1-hop neighbors,2-hop neighbors,3-hop neibors,4-hop neighbors,3-hop neighbors
O43741,144,3369,NaN,10575,8956.0
